In [11]:
from pathlib import Path

import pandas as pd
from icecream import colorize, ic

from backend.Database import Database

from icecream import ic, colorize

ic.configureOutput(outputFunction=lambda s: print(colorize(s) + "\n"))  # print debug
ic("bonjour");

ic| 'bonjour'



Creation dataframe

In [12]:
db = Database()
client = db.client


pipeline = [
    {
        "$lookup": {
            "from": "products",
            "localField": "product_id",
            "foreignField": "_id",
            "as": "orderdetails",
        }
    }
]

df = pd.DataFrame(list(client.poubelle.aggregate(pipeline)))
df

connexion to db established !



,_id,coef_touristes,densite,id_zone,gps,nextCollectionDate
0,6584ad110086777823b884cd,None,None,6584ad110086777823b884cc,"42°36'39""N 9°21'14""E",None
1,6584ad110086777823b884cf,None,None,6584ad110086777823b884cc,"42°36'44""N 9°21'24""E",None
2,6584ad110086777823b884d1,None,None,6584ad110086777823b884cc,"42°36'51""N 9°21'17""E",None
3,6584ad120086777823b884d3,None,None,6584ad110086777823b884cc,"42°36'55""N 9°21'05""E",None
4,6584ad120086777823b884d5,None,None,6584ad110086777823b884cc,"42°37'01""N 9°21'20""E",None
...,...,...,...,...,...,...
98,6584b2d80086777823b90635,None,None,6584b2d70086777823b90620,"42°41'52""N 9°21'08""E",None
99,6584b3900086777823b915e7,None,None,6584b3900086777823b915e6,"42°39'36""N 9°14'33""E",None
100,6584b3a00086777823b91757,None,None,6584b3a00086777823b91756,"42°35'53""N 9°15'53""E",None
101,6584b3a10086777823b91759,None,None,6584b3a00086777823b91756,"42°35'54""N 9°16'02""E",None


In [13]:
df = df.drop(columns=["coef_touristes", "gps", "nextCollectionDate", "densite"])
df

,_id,id_zone
0,6584ad110086777823b884cd,6584ad110086777823b884cc
1,6584ad110086777823b884cf,6584ad110086777823b884cc
2,6584ad110086777823b884d1,6584ad110086777823b884cc
3,6584ad120086777823b884d3,6584ad110086777823b884cc
4,6584ad120086777823b884d5,6584ad110086777823b884cc
...,...,...
98,6584b2d80086777823b90635,6584b2d70086777823b90620
99,6584b3900086777823b915e7,6584b3900086777823b915e6
100,6584b3a00086777823b91757,6584b3a00086777823b91756
101,6584b3a10086777823b91759,6584b3a00086777823b91756


In [ ]:
df[]